In [16]:
# ----------
# User Instructions:
# 
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, your function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, your function should return the string
# 'fail'
# ----------

# Grid format:
#   0 = Navigable space
#   1 = Occupied space

grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0]]
grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0]]
init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']


In [17]:
def find_route(expansion_grid, final_loc, ):
    route_grid  = [['' for col in range(len(expansion_grid[0]))] \
                   for row in range(len(expansion_grid))]
    

def find_route2(expansion_grid, tracked_locs, delta, delta_name, success=True):
    route_grid  = [[' ' for col in range(len(expansion_grid[0]))] \
                   for row in range(len(expansion_grid))]
    
    # curr_loc = last_loc
    tracked_locs = sorted(tracked_locs, reverse=True)
    curr_loc = [tracked_locs[0][1],tracked_locs[0][2]]
    if success:
        min_cost_route = [['*',tracked_locs[0][1],tracked_locs[0][2]]]
    else:
        min_cost_route = [['#',tracked_locs[0][1],tracked_locs[0][2]]]
    tracked_locs.pop(0)
    while len(tracked_locs):
        filtered_locs = [x for x in tracked_locs if x[0] == tracked_locs[0][0]]
        for loc in filtered_locs:
            min_cost = 1e10
            min_cost_locs = []
            for i,d in enumerate(delta):
                d_x, d_y = d
                calc_x, calc_y = loc[1] + d_x, loc[2] + d_y
                if calc_x == curr_loc[0] and calc_y == curr_loc[1]:
                    min_cost_locs.append([loc[0],delta_name[i], loc[1], loc[2]])
            if min_cost_locs: 
                curr_loc = min(min_cost_locs)[2:]
                min_cost_route.append(min(min_cost_locs)[1:])
        for i in range(len(filtered_locs)): tracked_locs.pop(0)

    for i,loc in enumerate(min_cost_route):
        # if i == 0:
        #     if success: 
        #         route_grid[loc[1]][loc[2]] = "*"
        #     else:
        #         route_grid[loc[1]][loc[2]] = "#"
        # else:
        route_grid[loc[1]][loc[2]] = loc[0]
    return route_grid
            
        
            
        
        
        
        
            

In [18]:
def next_loc(open_list, delta, cost):
    new_open_list = []
    curr_loc = open_list[0][1:]
    
    g_new = open_list[0][0] + cost
    
    for new_loc in delta:
        new_open_list.append([g_new, new_loc[0] + curr_loc[0], new_loc[1] + curr_loc[1]])
    return new_open_list

def prettyl(nlist, heading=""):
    out = ""
    if heading: 
        out += f"{heading}\n"
    for x in nlist:
        out += f"\t{x}\n"
    out += "-"*5 + "\n"
    
    return out

def search(grid, init, goal,cost, delta, delta_name, debug = False):
    open_list = [[0, *init]]
    if debug: print(f"intial open list:\n{prettyl(open_list)}")

    searched = []

    itr = 0
    found = False
    expansion_grid = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    expand_count = 0
    # expansion_grid[init[0]][init[1]] = expand_count
    # expand_count += 1

    tracked_locs = []
    
    while(len(open_list)):

        print(f"take list item\n{open_list[0]}")

        # update expansion_grid
        expansion_grid[open_list[0][1]][open_list[0][2]] = expand_count
        expand_count += 1
        
        new_open_list = next_loc(open_list, delta, cost)
        # filter open_list
        # ids to remove
        remove_ids = []
        for idn ,x in enumerate(new_open_list):
            i,j = x[1:]
            # check using limits of map
            if i < 0 or i >= len(grid):
                remove_ids.append(idn)
            elif j < 0 or j >= len(grid[0]):
                remove_ids.append(idn)
            # obstacle
            elif grid[i][j] == 1:
                remove_ids.append(idn)
            # already matched
            elif x[1:] in searched:
                # print(f"{x} in searched")
                remove_ids.append(idn)
            else:
                continue
        new_open_list = [new_open_list[i] for i in range(len(new_open_list)) if i not in remove_ids]
                
        # move opened to searched
        tracked_locs.append(open_list[0])
        searched.append(open_list.pop(0)[1:])
        # print(f"{searched=}")

        # check whether goal in new_list
        for x in new_open_list:
            if x[1:] == goal:
                # print("### Search success")
                expansion_grid[x[1]][x[2]] = expand_count
                # print(f"{prettyl(expansion_grid)}")
                found = x

                ## when found, create route grid
                # route_grid = find_route(expansion_grid, x, found=True)
                tracked_locs.append(x)
                # route_grid = find_route2(expansion_grid, tracked_locs, 
                                         # delta, delta_name, success=True)
                # print(f"{prettyl(route_grid)}")
                break
        if found: 
            break

        
        # sort by g

        for loc in new_open_list:
            if loc in open_list:
                continue
            open_list.append(loc)
        # open_list += new_open_list
        open_list.sort() # automatically sorts by first element in inner list
  
                
        if debug: print(f"new open list:\n{prettyl(open_list)}")

        # itr += 1
    if not found:
        print("fail")
        print(f"{prettyl(expansion_grid, heading='Expansion grid')}")
        route_grid = find_route2(expansion_grid, tracked_locs,
                                delta, delta_name, success=False)
        print(f"{prettyl(route_grid, heading='Route grid')}")
    else:
        print(f"{prettyl(expansion_grid, heading='Expansion grid')}")
        route_grid = find_route2(expansion_grid, tracked_locs, 
                                         delta, delta_name, success=True)
        print(f"{prettyl(route_grid, heading='Route grid')}")
        return found
        

        

    
    
        

In [19]:
search(grid, init, goal, cost, delta, delta_name, debug=True)

intial open list:
	[0, 0, 0]
-----

take list item
[0, 0, 0]
new open list:
	[1, 0, 1]
	[1, 1, 0]
-----

take list item
[1, 0, 1]
new open list:
	[1, 1, 0]
	[2, 1, 1]
-----

take list item
[1, 1, 0]
new open list:
	[2, 1, 1]
	[2, 2, 0]
-----

take list item
[2, 1, 1]
new open list:
	[2, 2, 0]
	[3, 2, 1]
-----

take list item
[2, 2, 0]
new open list:
	[3, 2, 1]
	[3, 3, 0]
-----

take list item
[3, 2, 1]
new open list:
	[3, 3, 0]
	[4, 3, 1]
-----

take list item
[3, 3, 0]
new open list:
	[4, 3, 1]
	[4, 4, 0]
-----

take list item
[4, 3, 1]
new open list:
	[4, 4, 0]
	[5, 4, 1]
-----

take list item
[4, 4, 0]
new open list:
	[5, 4, 1]
-----

take list item
[5, 4, 1]
new open list:
-----

fail
Expansion grid
	[0, 1, -1, -1, -1, -1]
	[2, 3, -1, -1, -1, -1]
	[4, 5, -1, -1, -1, -1]
	[6, 7, -1, -1, -1, -1]
	[8, 9, -1, -1, -1, -1]
-----

Route grid
	['v', ' ', ' ', ' ', ' ', ' ']
	['v', ' ', ' ', ' ', ' ', ' ']
	['v', ' ', ' ', ' ', ' ', ' ']
	['v', ' ', ' ', ' ', ' ', ' ']
	['>', '#', ' ', ' ',

## Test area